In [1]:
#!pip install tweepy
#!pip install langdetect

In [2]:
import pandas as pd
import langdetect as ld
from datetime import datetime


In [3]:
keys=[]
with open("./access_keys.txt", encoding="utf8") as fin:
    for line in fin.readlines():
            text = line.strip()
            keys.append(text)

In [4]:
import tweepy
#IMPORTANT: MUST DELETE THESE HARDCODED KEYS
auth = tweepy.OAuthHandler(keys[0], keys[1])
auth.set_access_token(keys[2], keys[3])

api = tweepy.API(auth)

In [5]:
def searchByHashtag(words, date_since, date_until, number=10, filename='scraped_tweets.csv'):
    #db = pd.DataFrame(columns=['username', 'description', 'location', 'likes','retweets', 'following',
    #                           'followers', 'totaltweets', 'text', 'hashtags', 'date'])
    #tweets = tweepy.Cursor(api.search, q=words, lang="en", since=date_since, until=date_until, tweet_mode="extended").items(number)
    list_tweets = api.search_full_archive("CovidTweeting", query=words, fromDate=date_since, toDate=date_until,
                                             maxResults=number)
    #list_tweets = [tweet for tweet in tweets]
    total_tweets = []
    i = 1
    for tweet in list_tweets:
        username = tweet.user.screen_name
        description = tweet.user.description
        location = tweet.user.location
        likes = tweet.favorite_count
        retweetcount = tweet.retweet_count
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        hashtags = tweet.entities['hashtags']
        date = tweet.created_at
          
        # Retweets can be distinguished by a retweeted_status attribute,
        # in case it is an invalid reference, except block will be executed
        try:
            text = tweet.retweeted_status.text
        except AttributeError:
            text = tweet.text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
          
        # Here we are appending all the extracted information in the DataFrame
        ith_tweet = [username, description, location, likes, retweetcount, following,
                     followers, totaltweets, text, hashtext, date]
        #db.loc[len(db)] = ith_tweet
          
        i = i+1
        total_tweets.append(ith_tweet)
    return total_tweets
    # we will save our database as a CSV file.
    #db.to_csv(filename)

First Pull

In [6]:
dates=[["202003100000", "202003220000"], ["202005100000", "202005220000"], ["202006100000", "202006220000"]]

In [7]:
def getResults(dates):
    searchResults = []
    for date in dates:
        lst = searchByHashtag("Corona OR Covid-19 OR Coronavirus", date[0], date[1], 100)
        flat_list = [item for sublist in lst for item in sublist]
        searchResults.append(lst)
    flat_search = [item for sublist in searchResults for item in sublist]
    return flat_search

In [9]:
df = pd.DataFrame(getResults(dates), columns=['username', 'description', 'location', 'likes','retweets', 'following',
                               'followers', 'totaltweets', 'text', 'hashtags', 'date'])

TweepError: {'message': 'Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.', 'sent': '2021-05-06T20:22:23+00:00', 'transactionId': '008db8a600180de5'}

In [75]:
df.to_csv("first_pull.csv")

Clean non-english tweets

In [3]:
df = pd.read_csv("first_pull.csv")
df

,Unnamed: 0,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date
0,0,SAGLCH,Just another funny guy trying to make it throu...,NaN,0,0,281,79,6440,#16 is my immune system fending off corona htt...,[],2020-03-21 23:59:59
1,1,504bayou,MEd Strategic planner for innovation & continu...,NaN,0,0,2597,983,22272,Everyone knows that we're facing a real crisis...,[],2020-03-21 23:59:59
2,2,Flaurilox,Run-of-the-mill atheist. Lover of mountains a...,Canada,0,0,5001,3076,37484,"""She tested for coronavirus, and her results w...",[],2020-03-21 23:59:59
3,3,howestreet,"Leading Financial News Source: Podcasts, Inves...",Vancouver,0,0,969,1345,20897,The Goddard Report with host @jimgoddard3 \n\n...,"['CoronaVirus', 'ToxicDrugs', 'FakeNews']",2020-03-21 23:59:59
4,4,hervanka,NaN,NaN,0,0,18,12,1394,@Yassarpurwa @detikcom Masalah itu obaf belum ...,[],2020-03-21 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,ymmontoya,"Amante del espacio, la cocina y los gatos; Adm...",Colombia,0,0,746,273,6424,Reporte oficial casos positivos de Coronavirus...,[],2020-06-21 23:59:53
296,296,8946fc64e5bd49c,"Comunicadora social, periodista, mujer q lucha...",NaN,1,0,537,153,4691,@TataQuispe Sr Quispe debe quedarse en su casa...,[],2020-06-21 23:59:53
297,297,Skanlyn123,NaN,NaN,0,0,100,4,953,@NeverGetGot @Atlas_Shoulders @Pr1ncessLily @C...,[],2020-06-21 23:59:53
298,298,RosendoSalgadoV,Delegado Presidente del Comité Ejecutivo Estat...,"Durango, México",0,0,107,463,22825,El 28 de febrero pasado se detectó el primer c...,[],2020-06-21 23:59:53


In [5]:
def getLangs(df):
    langs = []
    for text in df['text']:
        l = ""
        try:
            l = ld.detect(text)
        except:
            l = "error"
        langs.append(l)
    df['langs'] = langs

In [14]:
df['langs'] = langs

In [17]:
df = df[df['langs']=="en"]

In [18]:
df.to_csv("first_pull.csv")

In [24]:
dates=[["202007010000", "202007220000"], ["202008100000", "202008220000"], 
       ["202009100000", "202009220000"], ["202010100000", "202010220000"],
       ["202011100000", "202012220000"]]

In [27]:
df2 = getResults(dates)

In [29]:
df2 = pd.DataFrame(df2, columns=['username', 'description', 'location', 'likes','retweets', 'following',
                               'followers', 'totaltweets', 'text', 'hashtags', 'date'])

In [31]:
getLangs(df2)

In [33]:
df2 = df2[df2['langs']=="en"]

In [34]:
df2

,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date,langs
0,OrlandoForRev,Committed to the needs of working people and t...,"Orlando, FL",0,0,922,521,5538,"“Unfortunately, now Florida is in a little tou...",[],2020-07-21 23:59:59,en
3,shirodgers,None,None,0,0,191,107,85895,No White House coronavirus task force members ...,[],2020-07-21 23:59:59,en
5,Badominicana,Add me On #Snapchat and let's start a #Streak ...,"New York , New York",0,0,1375,1173,130455,Chinese hackers charged in theft of trade secr...,[],2020-07-21 23:59:59,en
6,iOptimizeRealty,"Tenant Representation, Economic Development, S...","Commack, NY",0,0,2926,2240,64306,COVID-19 &amp; the Force Majeure Clause in Lea...,"[CRE, ForceMajeure, COVID]",2020-07-21 23:59:59,en
8,RoyKenagy,"Student, reader, librarian, consultant, teacher.",Des Moines,0,0,1070,1948,93274,Opinion | Where Is the Outrage? Americans are ...,[],2020-07-21 23:59:59,en
...,...,...,...,...,...,...,...,...,...,...,...,...
492,JacqueBlue,Resister. Artist. Blind dyslexic that makes nu...,"North Carolina, USA",0,0,2483,2477,9598,No Bible verse is going to save Marco Rubio fr...,[],2020-12-21 23:59:54,en
493,7Willow33,"Phillydelphia girl🥨 in New England,lost Mom to...",Eastern CT shoreline...,0,0,442,156,48613,Supreme Court Justices Can Get Vaccinated Now....,[],2020-12-21 23:59:54,en
495,PhillinThe__,Grad Student | Father | GVSU Alumnus | 1906,Where the money reside,0,0,633,826,44496,Jairo Castillo was climbing the ranks in the s...,[],2020-12-21 23:59:54,en
497,sloaneellen,Arizona Native. Adopted. Football and Baseball...,Arizona,0,0,5003,1088,117303,NBC News confirms: The United States is now re...,[],2020-12-21 23:59:54,en


In [35]:
df2.to_csv("second_pull.csv")

In [5]:
dates = [["202003011200", "202003100000"], ["202003101200", "202003150000"], ["202003161200", "202003200000"], ["202003211200", "202003300000"]]

In [9]:
results = getResults(dates)

In [10]:
df3 = pd.DataFrame(results, columns=['username', 'description', 'location', 'likes','retweets', 'following',
                               'followers', 'totaltweets', 'text', 'hashtags', 'date'])

In [11]:
df3

,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date
0,bobotringan,Kadang jadi penggemar #ManUtd | \nBukan akun ...,Ubun ubunmu,0,0,703,3426,29691,Coronavirus deaths\n\nChina: 3119\nItaly: 463\...,[],2020-03-09 23:59:59
1,CherokeeNative3,Retired Paralegal standing in solidarity w/ACL...,92058,0,0,4069,6621,589979,The mob boss doesn't want an accurate count of...,[coronavirus],2020-03-09 23:59:59
2,Charo_Ocejo,No me gusta ir por la vida ofendiéndome por to...,España,0,0,1234,6122,163362,Los de Bilbao tenemos nuestros métodos para co...,[],2020-03-09 23:59:59
3,one4REALchange,"And sometimes, against all odds, against all l...",None,1,0,22108,22166,16513,@SenSchumer @JohnGunter2020 Rather than spend ...,[coronavirus],2020-03-09 23:59:59
4,cherrybaysb,"Mom, wife, grandmother to a yorkie named Belle...",None,0,0,2509,2460,395799,Schumer: What was glaringly missing from Presi...,[],2020-03-09 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...
395,BaCkFiRe,None,None,0,0,260,22,1538,La ONG del reconocido chef español José Andrés...,[],2020-03-29 23:59:59
396,Scorpiothingzz,"she/her , Book lover 📚, Scorpio ♏️ insta-tiny_...","Brooklyn, NY",0,0,392,39,5186,A sad scene at Brooklyn Hospital #Covid_19 #co...,"[Covid_19, coronavirus]",2020-03-29 23:59:59
397,Ichnusa67,"That which can be asserted without evidence, c...",Vacuum,0,0,889,74,6829,"Thomas Schaefer, ministro de finanzas de la re...",[],2020-03-29 23:59:59
398,Maiev__,None,None,0,0,495,201,14672,Certificado de defunción de persona fallecida ...,[],2020-03-29 23:59:59


In [14]:
getLangs(df3)

In [15]:
df3

,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date,langs
0,bobotringan,Kadang jadi penggemar #ManUtd | \nBukan akun ...,Ubun ubunmu,0,0,703,3426,29691,Coronavirus deaths\n\nChina: 3119\nItaly: 463\...,[],2020-03-09 23:59:59,en
1,CherokeeNative3,Retired Paralegal standing in solidarity w/ACL...,92058,0,0,4069,6621,589979,The mob boss doesn't want an accurate count of...,[coronavirus],2020-03-09 23:59:59,en
2,Charo_Ocejo,No me gusta ir por la vida ofendiéndome por to...,España,0,0,1234,6122,163362,Los de Bilbao tenemos nuestros métodos para co...,[],2020-03-09 23:59:59,et
3,one4REALchange,"And sometimes, against all odds, against all l...",None,1,0,22108,22166,16513,@SenSchumer @JohnGunter2020 Rather than spend ...,[coronavirus],2020-03-09 23:59:59,en
4,cherrybaysb,"Mom, wife, grandmother to a yorkie named Belle...",None,0,0,2509,2460,395799,Schumer: What was glaringly missing from Presi...,[],2020-03-09 23:59:59,en
...,...,...,...,...,...,...,...,...,...,...,...,...
395,BaCkFiRe,None,None,0,0,260,22,1538,La ONG del reconocido chef español José Andrés...,[],2020-03-29 23:59:59,es
396,Scorpiothingzz,"she/her , Book lover 📚, Scorpio ♏️ insta-tiny_...","Brooklyn, NY",0,0,392,39,5186,A sad scene at Brooklyn Hospital #Covid_19 #co...,"[Covid_19, coronavirus]",2020-03-29 23:59:59,en
397,Ichnusa67,"That which can be asserted without evidence, c...",Vacuum,0,0,889,74,6829,"Thomas Schaefer, ministro de finanzas de la re...",[],2020-03-29 23:59:59,es
398,Maiev__,None,None,0,0,495,201,14672,Certificado de defunción de persona fallecida ...,[],2020-03-29 23:59:59,es


In [16]:
df3 = df3[df3['langs']=="en"]

In [18]:
df3.to_csv("thirdpull_earlyLockdown.csv")

In [19]:
dates = [["202007011200", "202007100000"], ["202007101200", "202007150000"], ["202007161200", "202007200000"], ["202007211200", "202007300000"]]

In [20]:
results = getResults(dates)

In [21]:
df4 = pd.DataFrame(results, columns=['username', 'description', 'location', 'likes','retweets', 'following',
                               'followers', 'totaltweets', 'text', 'hashtags', 'date'])

In [22]:
df4

,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date
0,donatelo31,Publicista bella e inteligente viajera x el mu...,En algun lugar del mundo...,0,0,2059,694,57739,Muchas gracias y bendiciones a todos! Hoy fue ...,[],2020-07-09 23:59:59
1,REoptimizer,REoptimizer® is the world's first and most eff...,"Commack, NY",0,0,3870,4579,124890,Roadmap to Returning to Your Office Post-COVID...,"[CRE, COVID, NewNormal, Office, RoadMap]",2020-07-09 23:59:59
2,mazrnow,Cloud Solutions Architect who believes technol...,Canada,0,0,309,219,3766,"Avec le Covid-19, les clouds français devienne...","[CloudComputing, ObjexTech]",2020-07-09 23:59:59
3,zzmar1,🧘🏻,"Sorocaba, Brasil",1,0,382,118,998,me dói lembrar que estamos em julho e que eu p...,[],2020-07-09 23:59:59
4,Groovy_Ky,Realtor for Better Homes and Gardens Real Esta...,Maryland,0,0,5289,5121,596835,NFL teams will be forbidden from postgame inte...,[],2020-07-09 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...
395,MeghanShreve,"Mom to M & H, Wifey to @bshreve34, eater of al...","Lawrence, KS",0,0,2284,1081,13434,A state law taking effect Friday will restrict...,[],2020-07-29 23:59:56
396,PBreckenfeld,💢 Eng. Civil,"São João do Rio do Peixe, PB",2,0,306,150,1421,Mirassol perdeu um time completo por conta do ...,[],2020-07-29 23:59:55
397,TheeKweenK,𝙼𝚞𝚟𝚊|#𝙵𝙰𝙼𝚄|𝙲𝚛𝚎𝚊𝚝𝚒𝚟𝚎|𝙷𝚎𝚛𝚋𝚊𝚕 & 𝙰𝚜𝚝𝚛𝚘 𝚂𝚊𝚟𝚊𝚗𝚝 •𝚂𝚘𝚞...,"Atlanta, GA",0,0,4072,4219,89814,Remember someone tweeted to watch what big com...,[],2020-07-29 23:59:55
398,gineilagh,21•C3 TM👩🏻‍⚕️,Panamá,0,0,524,1031,7081,Gracias a Dios mi mamá hoy venció al covid-19 ...,[],2020-07-29 23:59:55


In [23]:
getLangs(df4)

In [24]:
df4

,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date,langs
0,donatelo31,Publicista bella e inteligente viajera x el mu...,En algun lugar del mundo...,0,0,2059,694,57739,Muchas gracias y bendiciones a todos! Hoy fue ...,[],2020-07-09 23:59:59,es
1,REoptimizer,REoptimizer® is the world's first and most eff...,"Commack, NY",0,0,3870,4579,124890,Roadmap to Returning to Your Office Post-COVID...,"[CRE, COVID, NewNormal, Office, RoadMap]",2020-07-09 23:59:59,en
2,mazrnow,Cloud Solutions Architect who believes technol...,Canada,0,0,309,219,3766,"Avec le Covid-19, les clouds français devienne...","[CloudComputing, ObjexTech]",2020-07-09 23:59:59,fr
3,zzmar1,🧘🏻,"Sorocaba, Brasil",1,0,382,118,998,me dói lembrar que estamos em julho e que eu p...,[],2020-07-09 23:59:59,pt
4,Groovy_Ky,Realtor for Better Homes and Gardens Real Esta...,Maryland,0,0,5289,5121,596835,NFL teams will be forbidden from postgame inte...,[],2020-07-09 23:59:59,en
...,...,...,...,...,...,...,...,...,...,...,...,...
395,MeghanShreve,"Mom to M & H, Wifey to @bshreve34, eater of al...","Lawrence, KS",0,0,2284,1081,13434,A state law taking effect Friday will restrict...,[],2020-07-29 23:59:56,en
396,PBreckenfeld,💢 Eng. Civil,"São João do Rio do Peixe, PB",2,0,306,150,1421,Mirassol perdeu um time completo por conta do ...,[],2020-07-29 23:59:55,pt
397,TheeKweenK,𝙼𝚞𝚟𝚊|#𝙵𝙰𝙼𝚄|𝙲𝚛𝚎𝚊𝚝𝚒𝚟𝚎|𝙷𝚎𝚛𝚋𝚊𝚕 & 𝙰𝚜𝚝𝚛𝚘 𝚂𝚊𝚟𝚊𝚗𝚝 •𝚂𝚘𝚞...,"Atlanta, GA",0,0,4072,4219,89814,Remember someone tweeted to watch what big com...,[],2020-07-29 23:59:55,en
398,gineilagh,21•C3 TM👩🏻‍⚕️,Panamá,0,0,524,1031,7081,Gracias a Dios mi mamá hoy venció al covid-19 ...,[],2020-07-29 23:59:55,es


In [25]:
df4 = df4[df4['langs']=="en"]

In [26]:
df4.to_csv("fourthpull_midLockdown.csv")

Get more dates more better...

In [6]:
def generateDates(date1, date2):
    return pd.date_range(date1, date2).tolist()

In [36]:
dates = generateDates("2020-05-01", "2020-05-20")

In [37]:
newDates = []
for i in range(0, 20, 2):
    newDates.append([dates[i].strftime("%Y%m%d%H%M"), dates[i+1].strftime("%Y%m%d%H%M")])

In [38]:
newResults = getResults(newDates)

In [39]:
df5 = pd.DataFrame(newResults, columns=['username', 'description', 'location', 'likes','retweets', 'following',
                               'followers', 'totaltweets', 'text', 'hashtags', 'date'])

In [40]:
df5

,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date
0,snarky_op,COUNTRY B4 PARTY⚖️🌏\nRussian-Orthodox-LifeLong...,"North Logan, UT",1,1,5000,3650,278356,"@LouDobbs @realDonaldTrump @KimStrassel ""Citin...",[],2020-05-01 23:59:59
1,J_AAngelVill1,Mi Pasión Fútbol Fan @NattiNatasha 🇩🇴 @LadySh...,México,0,0,333,1025,80255,"Preguntas y respuestas en vivo, Contra el COVI...",[TelevisaTeAcompaña],2020-05-01 23:59:59
2,malleeabull,None,None,0,0,124,8,78,Thanks @carlzimmer and @13pt for this detailed...,[],2020-05-01 23:59:59
3,MY_KINDA_SEX_ED,★ they/them disabled Irish/Canadian giving the...,Canada,0,0,1187,1088,68429,'I died and came back': 12-year-old recovers f...,[],2020-05-01 23:59:59
4,207_Melissa,Lifetime in 207 🇺🇸 Happily Married ❤️ 5 amazin...,Gorham Maine,0,0,4129,3426,113110,"If you must travel, take these steps to protec...",[COVID19],2020-05-01 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...
995,katertomater,Gregarious introvert. Idealistic believer in t...,None,0,0,3903,2386,25828,Finally some potentially positive news: a new...,[],2020-05-19 23:59:57
996,xinxolinha,Gosto do Macron porque ele faz fascista entreg...,World,0,0,153,557,23375,@O_Pensador64 @benedictoleonar @folha Do que c...,[],2020-05-19 23:59:57
997,IamNurseTrish,If U R offended cuz I require a vaccine passpo...,The right side of history!,0,0,10484,16638,68739,The scientist who created Florida's COVID-19 d...,[],2020-05-19 23:59:57
998,bigjason252,"Sou PT ,Sou LulaLivre,LulaPresidente... Odeio ...",None,0,0,1721,1501,304534,Mortes nas últimas 24 horas por covid-19:\n\nB...,[],2020-05-19 23:59:57


In [41]:
dates_june = generateDates("2020-06-01", "2020-06-30")

In [44]:
newDates_june = []
for i in range(0, 30, 2):
    newDates_june.append([dates_june[i].strftime("%Y%m%d%H%M"), dates_june[i+1].strftime("%Y%m%d%H%M")])

In [46]:
len(newDates_june)

15

In [47]:
newResults_june = getResults(newDates_june)

In [48]:
df6 = pd.DataFrame(newResults_june, columns=['username', 'description', 'location', 'likes','retweets', 'following',
                               'followers', 'totaltweets', 'text', 'hashtags', 'date'])

In [49]:
df6

,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date
0,SolisSantos,"Director de Oye Veracruz, Diario del Golfo de ...","Xalapa, Veracruz",0,0,4791,2473,17608,La ilegalidad presidencial en tiempos del coro...,[],2020-06-01 23:59:59
1,jordantcrawford,Freelance content creator | Bad at video games...,"Atlanta, GA",0,0,157,67,857,For those keeping score at home:\n\nThe Presid...,[],2020-06-01 23:59:59
2,brianjameslive,Radio host and author. Follow for “feel good” ...,The United States of America,1,1,3181,5316,78722,Legislation needs to be introduced to fast-tra...,[],2020-06-01 23:59:59
3,RayGiles1,"Editor of the blog ""Legal Threats Facing Donal...","Roseville, CA",0,0,2,240,66473,"@Post_AG During the coronavirus crisis, Presid...",[],2020-06-01 23:59:59
4,meaniesbeanie,"proud @Wellesley alum, dog mom, lawyer, news e...","Boston, MA",0,0,903,185,2019,"In an opinion piece for @CNN this week, @DrPau...",[],2020-06-01 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...
1495,rach_rabbit,None,None,0,0,132,73,7522,American Airlines is recklessly endangering mi...,[],2020-06-29 23:59:55
1496,thunderbolt5523,#IAmCanadian #TrumpMustGo #GunControl #NoSport...,"Ontario, Canada",1,0,4854,4289,42897,@Mike_Pence Uhhh isn’t the Supreme Court suppo...,[RussianBounty],2020-06-29 23:59:55
1497,TransitoRos,"Información del tránsito, calles, accesos, rut...",Rosario,0,0,25842,47879,89482,"Situación actual de los casos descartados, con...",[Coronavirus],2020-06-29 23:59:55
1498,NejdetKaanAksu,"Mustafa Kemal Atatürk’ün “Fikri hür, Vicdanı h...","Mordor , Mount Doom",7,0,2505,2713,9865,Duramaz. Beslenme ve barınma ihtiyaçları baş g...,[],2020-06-29 23:59:55


In [11]:
def getResultsByMonth(dates):
    newDates = []
    for i in range(0, len(dates), 2):
        newDates.append([dates[i].strftime("%Y%m%d%H%M"), dates[i+1].strftime("%Y%m%d%H%M")])
    newResults = getResults(newDates)
    return newResults

In [12]:
dates_July = generateDates("2020-07-01", "2020-07-30")
df7 = getResultsByMonth(dates_July)

NameError: name 'generateDates' is not defined

In [52]:
df6.to_csv("June_data.csv")

In [53]:
df5.to_csv("May_data.csv")